# Breast Cancer Classifier – Overfitting Mitigation (v2)
This notebook includes detailed precision, recall, F1-score, and confusion matrix reporting.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, precision_recall_curve)
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Custom recall metric
recall_metric = tf.keras.metrics.Recall(name='recall')


## 1. Load & Preprocess Data

In [2]:
data = pd.read_csv('Breast_cancer_data.csv')
X = data.drop('diagnosis', axis=1).values
y = data['diagnosis'].values
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)
scaler = StandardScaler()
X_trainval = scaler.fit_transform(X_trainval)
X_test = scaler.transform(X_test)


## 2. Stratified K-Fold Cross-Validation & Training

In [3]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_metrics = []
for fold, (train_idx, val_idx) in enumerate(kf.split(X_trainval, y_trainval), 1):
    # Prepare fold data
    X_tr, X_val = X_trainval[train_idx], X_trainval[val_idx]
    y_tr, y_val = y_trainval[train_idx], y_trainval[val_idx]
    # Oversample malignant twice
    df = pd.DataFrame(X_tr)
    df['label'] = y_tr
    df_maj = df[df.label==0]
    df_min = df[df.label==1]
    df_min_up = resample(df_min, replace=True, n_samples=len(df_maj)*2, random_state=42)
    df_fold = pd.concat([df_maj, df_min_up])
    X_tr_up = df_fold.drop('label',axis=1).values
    y_tr_up = df_fold['label'].values
    # Build model
    def build_model(input_dim):
        model = Sequential([
            Dense(128, activation=None, input_shape=(input_dim,), kernel_regularizer=regularizers.l2(1e-4)),
            BatchNormalization(), Activation('relu'), Dropout(0.4),
            Dense(64, activation=None, kernel_regularizer=regularizers.l2(1e-4)), BatchNormalization(), Activation('relu'), Dropout(0.3),
            Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4)), Dropout(0.2),
            Dense(1, activation='sigmoid')
        ])
        model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            metrics=['accuracy', recall_metric]
        )
        return model
    model = build_model(X_tr_up.shape[1])
    # Train
    callbacks = [EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                 ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)]
    history = model.fit(X_tr_up, y_tr_up, validation_data=(X_val,y_val), epochs=100, batch_size=32, callbacks=callbacks, verbose=0)
    # Evaluate on validation
    y_val_pred = (model.predict(X_val).ravel() >= 0.5).astype(int)
    prec = precision_score(y_val, y_val_pred)
    rec = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)
    fold_metrics.append((prec, rec, f1))
    print(f'Fold {fold}: Precision={prec:.3f}, Recall={rec:.3f}, F1={f1:.3f}')
print('Avg Precision:', np.mean([m[0] for m in fold_metrics]))
print('Avg Recall:',    np.mean([m[1] for m in fold_metrics]))
print('Avg F1:',        np.mean([m[2] for m in fold_metrics]))


C:\Users\sebas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Fold 1: Precision=0.862, Recall=0.982, F1=0.918


C:\Users\sebas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Fold 2: Precision=0.982, Recall=0.982, F1=0.982


C:\Users\sebas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/stepWARNING:tensorflow:6 out of the last 9 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000207669C7420> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Fold 3: Precision=0.948, Recall=0.965, F1=0.957


C:\Users\sebas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Fold 4: Precision=0.964, Recall=0.947, F1=0.956


C:\Users\sebas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Fold 5: Precision=0.932, Recall=0.965, F1=0.948
Avg Precision: 0.9377519136149054
Avg Recall: 0.968421052631579
Avg F1: 0.9522077481649809


## 3. Final Training & Detailed Test Evaluation

In [4]:
# Retrain model on full training data with 2× oversampling
df_full = pd.DataFrame(X_trainval); df_full['label']=y_trainval
df_full_maj = df_full[df_full.label==0]; df_full_min = df_full[df_full.label==1]
df_full_min_up = resample(df_full_min, replace=True, n_samples=len(df_full_maj)*2, random_state=42)
df_full_up = pd.concat([df_full_maj, df_full_min_up])
X_full_up = df_full_up.drop('label',axis=1).values; y_full_up = df_full_up['label'].values
model = build_model(X_full_up.shape[1])
model.fit(X_full_up, y_full_up, validation_split=0.2, epochs=100, batch_size=32, callbacks=callbacks, verbose=1)
# Test evaluation
y_test_prob = model.predict(X_test).ravel()
y_test_pred = (y_test_prob >= 0.5).astype(int)
print('Test Metrics:')
print('Accuracy:', accuracy_score(y_test, y_test_pred))
print('Precision:', precision_score(y_test, y_test_pred))
print('Recall:', recall_score(y_test, y_test_pred))
print('F1-score:', f1_score(y_test, y_test_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_test_pred))
print('Classification Report:\n', classification_report(y_test, y_test_pred))


Epoch 1/100


C:\Users\sebas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.5772 - loss: 0.7413 - recall: 0.8095 - val_accuracy: 0.9804 - val_loss: 0.5281 - val_recall: 0.9804 - learning_rate: 0.0010
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8146 - loss: 0.4530 - recall: 0.9034 - val_accuracy: 0.9804 - val_loss: 0.4311 - val_recall: 0.9804 - learning_rate: 0.0010
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8924 - loss: 0.3164 - recall: 0.9523 - val_accuracy: 0.9804 - val_loss: 0.3566 - val_recall: 0.9804 - learning_rate: 0.0010
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8836 - loss: 0.3215 - recall: 0.9494 - val_accuracy: 0.9804 - val_loss: 0.3134 - val_recall: 0.9804 - learning_rate: 0.0010
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8953 - loss: 0.2795 - recall: 0.9457 - val_accuracy: 0.9902 - val_loss: 0.2737 - val_recall: 0.9902 - learning_rate: 0.0010
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Test Metrics:
Accuracy: 0.90